# Revised case normalization for UZH 2019





In [1]:
import pandas as pd
import sys
sys.path.insert(0, '/home/jovyan/work')

import os

from py.global_configs import *
from py.normalize import normalize

from src.service import bfs_cases_db_service as bfs_db

from src.utils.dataframe_utils import remove_duplicated_chops

/home/jovyan/work/src/models/BfsCase.py:5: SAWarning: On class 'BfsCase', Column object 'drg_cost_weight' named directly multiple times, only one will be used: drg_cost_weight, effective_cost_weight. Consider using orm.synonym instead
  class BfsCase(Base):


In [2]:

file_info = FileInfo(
         os.path.join(ROOT_DIR, 'raw_data/USZ_2018-2019_20200730.xlsx'),
         'USZ',
         '2019',
         ['Gesamtauffällige_USZ_2019'])

print(file_info)



FileInfo(path='/home/jovyan/work/src/revised_case_normalization/raw_data/USZ_2018-2019_20200730.xlsx', hospital_name_db='USZ', year='2019', sheets=['Gesamtauffällige_USZ_2019'])


In [3]:
cols_to_rename = dict(COLUMNS_TO_RENAME)
# Replace 'admno' with 'fall nummer'
cols_to_rename.pop('admno')
cols_to_rename['fall nummer'] = CASE_ID_COL

df_revised_case_d2d = normalize(file_info, 0)

Read 150 cases for USZ 2019
TYPES:
case_id             string
patient_id          string
gender              string
age_years            int64
duration_of_stay     int64
pccl                 int64
drg                 string
old_pd              string
new_pd              string
bfs_code            string
added_icds          string
removed_icds        string
added_chops         string
removed_chops       string
dtype: object


2022-10-17 18:56:07.860 | DEBUG    | src.utils.dataframe_utils:_validate_icd_codes:56 - row 93: discarded ICDs after validation {''}
2022-10-17 18:56:07.861 | DEBUG    | src.utils.dataframe_utils:_validate_icd_codes:56 - row 99: discarded ICDs after validation {''}
2022-10-17 18:56:07.861 | DEBUG    | src.utils.dataframe_utils:_validate_icd_codes:56 - row 105: discarded ICDs after validation {''}
2022-10-17 18:56:07.862 | DEBUG    | src.utils.dataframe_utils:_validate_icd_codes:56 - row 108: discarded ICDs after validation {''}
2022-10-17 18:56:07.863 | DEBUG    | src.utils.dataframe_utils:_validate_icd_codes:56 - row 122: discarded ICDs after validation {''}
2022-10-17 18:56:07.865 | DEBUG    | src.utils.dataframe_utils:_validate_icd_codes:56 - row 143: discarded ICDs after validation {''}
2022-10-17 18:56:07.866 | DEBUG    | src.utils.dataframe_utils:_validate_icd_codes:56 - row 145: discarded ICDs after validation {''}
2022-10-17 18:56:07.868 | DEBUG    | src.utils.dataframe_utils:_

In [4]:
df_revised_case_d2d

,case_id,patient_id,gender,age_years,duration_of_stay,pccl,drg,old_pd,new_pd,bfs_code,added_icds,removed_icds,added_chops,removed_chops
0,6400282213,3ED8F10117FCF5F5,M,54,13,3,E06A,J4400,J4400,M200,[F058],[],[],[]
1,6400287076,55C5A17F71DADEA0,M,62,5,3,D62C,J068,J068,M800,"[I8028, C770, C787]",[],[],[]
2,6400306817,0150AC67E3196B63,W,74,36,3,G72C,M8088,M8088,M100,[R64],[],[],[]
3,6400316927,153929A139272585,W,79,6,3,B68C,G3531,G3531,M100,[R471],[],[],[]
4,6400278895,014A89BB816D9215,W,71,44,3,F75D,T827,T827,M200,[D508],[D648],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,6400305282,B2EFB1C7F864D6BE,M,54,4,2,F67B,R522,R522,M100,[],[],[],[]
146,6400306506,2BD4038A71FFAFCE,M,51,7,2,M60B,A410,A410,M200,"[R651, C795, C774, N184]",[],[],[]
147,6400307264,45BD6CAFB4B36938,W,84,14,2,J02B,L8924,L8924,M200,[B965],[],[],[]
148,6400308115,CC100ED2DB478B7E,M,46,8,2,H64B,K830,K830,M100,[K831],[],[],[]


# Match revised DtoD data with the bfs data from the database


In [5]:
# get the case_id from revised_case

revised_case_id = df_revised_case_d2d['case_id'].values
revised_case_id[0]

'6400282213'

In [6]:
revised_case_id_stripped = [case_id.lstrip('0') for case_id in revised_case_id]

In [7]:
# change the case_id the same format as bfs data e.g. '0041194601' 
# revised_case_id_pad0 = [case_id.zfill(10) for case_id in revised_case_id]
# revised_case_id_pad0[0]
# df_revised_case_d2d['case_id'] = revised_case_id_pad0

In [8]:
# match to the database
revised_case_db = bfs_db.get_bfs_cases_by_ids_no_pad0(revised_case_id_stripped)
revised_case_db.head()

,drg_cost_weight,aimedic_id,hospital_id,case_id,patient_id,age_years,age_days,gender,duration_of_stay,clinic_id,ventilation_hours,admission_weight,gestation_age,admission_date,admission_type,discharge_date,discharge_type,drg,adrg,pccl
0,2.583,340028,14,6400282213,3ED8F10117FCF5F5,54,0,M,13,4,0,0,0,2018-12-27,11,2019-01-09,00,E06A,E06,3
1,2.583,340029,14,6400282213,31327693C8667DB6,54,0,M,13,4,0,0,0,2018-12-27,11,2019-01-09,00,E06A,E06,3
0,0.398,350002,14,6400287076,55C5A17F71DADEA0,62,0,M,5,10,0,0,0,2019-02-10,06,2019-02-15,00,D62C,D62,3
1,0.398,350003,14,6400287076,E8042604E439007F,62,0,M,5,10,0,0,0,2019-02-10,06,2019-02-15,00,D62C,D62,3
0,0.448,370254,14,6400306817,0150AC67E3196B63,74,0,W,36,3,0,0,0,2019-05-08,01,2019-06-13,00,G72C,G72,3


In [9]:
# create a new column for case_id_stripped with '0' for joining to the revised cases
revised_case_db['case_id_stripped']=  revised_case_db['case_id'].apply(lambda x: x.lstrip('0')) 
revised_case_db.head()

,drg_cost_weight,aimedic_id,hospital_id,case_id,patient_id,age_years,age_days,gender,duration_of_stay,clinic_id,...,admission_weight,gestation_age,admission_date,admission_type,discharge_date,discharge_type,drg,adrg,pccl,case_id_stripped
0,2.583,340028,14,6400282213,3ED8F10117FCF5F5,54,0,M,13,4,...,0,0,2018-12-27,11,2019-01-09,00,E06A,E06,3,6400282213
1,2.583,340029,14,6400282213,31327693C8667DB6,54,0,M,13,4,...,0,0,2018-12-27,11,2019-01-09,00,E06A,E06,3,6400282213
0,0.398,350002,14,6400287076,55C5A17F71DADEA0,62,0,M,5,10,...,0,0,2019-02-10,06,2019-02-15,00,D62C,D62,3,6400287076
1,0.398,350003,14,6400287076,E8042604E439007F,62,0,M,5,10,...,0,0,2019-02-10,06,2019-02-15,00,D62C,D62,3,6400287076
0,0.448,370254,14,6400306817,0150AC67E3196B63,74,0,W,36,3,...,0,0,2019-05-08,01,2019-06-13,00,G72C,G72,3,6400306817


In [10]:
df_revised_case_d2d['case_id_stripped']=  df_revised_case_d2d['case_id'].apply(lambda x: x.lstrip('0')) 
df_revised_case_d2d.head()


,case_id,patient_id,gender,age_years,duration_of_stay,pccl,drg,old_pd,new_pd,bfs_code,added_icds,removed_icds,added_chops,removed_chops,case_id_stripped
0,6400282213,3ED8F10117FCF5F5,M,54,13,3,E06A,J4400,J4400,M200,[F058],[],[],[],6400282213
1,6400287076,55C5A17F71DADEA0,M,62,5,3,D62C,J068,J068,M800,"[I8028, C770, C787]",[],[],[],6400287076
2,6400306817,0150AC67E3196B63,W,74,36,3,G72C,M8088,M8088,M100,[R64],[],[],[],6400306817
3,6400316927,153929A139272585,W,79,6,3,B68C,G3531,G3531,M100,[R471],[],[],[],6400316927
4,6400278895,014A89BB816D9215,W,71,44,3,F75D,T827,T827,M200,[D508],[D648],[],[],6400278895


# Forming the normalized dataframe: Merge the selected revised case data with the matched from bfs data
  If there are multiple cases were found based on case_id, patient_id will be used to match once more  

In [11]:
# Check if there are multiple cases were found based on case_id, patient_id will be used to match once more
if revised_case_db.shape[0] <= len(revised_case_id):
    revised_case_d2d_subset = df_revised_case_d2d[COL_SUBSET_FROM_REVISED_CASES + ['case_id_stripped', 'gender', 'age_years', 'duration_of_stay', 'pccl', 'drg']]
    revised_case_normalized = revised_case_d2d_subset.merge(revised_case_db, how ='inner', on=['case_id_stripped', 'gender', 'age_years', 'duration_of_stay', 'pccl', 'drg'], suffixes=None)
    num_match_case_id = revised_case_normalized.shape[0]
else:
    revised_case_d2d_subset = df_revised_case_d2d[COL_SUBSET_FROM_REVISED_CASES + ['case_id_stripped', 'patient_id', 'gender', 'age_years', 'duration_of_stay', 'pccl', 'drg']]
    revised_case_normalized = revised_case_d2d_subset.merge(revised_case_db, how ='inner', on=['case_id_stripped', 'patient_id', 'gender', 'age_years', 'duration_of_stay', 'pccl', 'drg'])
    num_match_case_patient_id = revised_case_normalized.shape[0]
print(revised_case_normalized.shape)
revised_case_normalized.head()

(146, 28)


,old_pd,new_pd,bfs_code,added_icds,removed_icds,added_chops,removed_chops,case_id_stripped,patient_id,gender,...,age_days,clinic_id,ventilation_hours,admission_weight,gestation_age,admission_date,admission_type,discharge_date,discharge_type,adrg
0,J4400,J4400,M200,[F058],[],[],[],6400282213,3ED8F10117FCF5F5,M,...,0,4,0,0,0,2018-12-27,11,2019-01-09,00,E06
1,J068,J068,M800,"[I8028, C770, C787]",[],[],[],6400287076,55C5A17F71DADEA0,M,...,0,10,0,0,0,2019-02-10,06,2019-02-15,00,D62
2,M8088,M8088,M100,[R64],[],[],[],6400306817,0150AC67E3196B63,W,...,0,3,0,0,0,2019-05-08,01,2019-06-13,00,G72
3,G3531,G3531,M100,[R471],[],[],[],6400316927,153929A139272585,W,...,0,3,0,0,0,2019-07-03,01,2019-07-09,00,B68
4,T827,T827,M200,[D508],[D648],[],[],6400278895,014A89BB816D9215,W,...,0,4,0,0,0,2018-11-30,11,2019-01-13,00,F75


In [12]:
df_revised_case_normalized = revised_case_normalized[COLS_TO_OUTPUT]
assert len(df_revised_case_normalized.columns) == len(COLS_TO_OUTPUT)

df_revised_case_normalized.head()

,aimedic_id,case_id,patient_id,gender,age_years,duration_of_stay,pccl,drg,old_pd,new_pd,bfs_code,added_icds,removed_icds,added_chops,removed_chops
0,340028,6400282213,3ED8F10117FCF5F5,M,54,13,3,E06A,J4400,J4400,M200,[F058],[],[],[]
1,350002,6400287076,55C5A17F71DADEA0,M,62,5,3,D62C,J068,J068,M800,"[I8028, C770, C787]",[],[],[]
2,370254,6400306817,0150AC67E3196B63,W,74,36,3,G72C,M8088,M8088,M100,[R64],[],[],[]
3,382594,6400316927,153929A139272585,W,79,6,3,B68C,G3531,G3531,M100,[R471],[],[],[]
4,337849,6400278895,014A89BB816D9215,W,71,44,3,F75D,T827,T827,M200,[D508],[D648],[],[]


# Export the normalized revised cases with the neccessary columns to a csv file


In [13]:

hospital = file_info.hospital_name_db.replace(' ', '_') # replace spaces with '_' for the file name
hospital_year = f'{hospital}_{file_info.year}'

output_folder = os.path.join(ROOT_DIR, 'normalized_revision_data/')

if not os.path.exists(output_folder):
    os.mkdir(output_folder)

output_path = os.path.join(ROOT_DIR, 'normalized_revision_data/') + hospital_year + '.csv'

df_revised_case_normalized.to_csv(output_path)


# Report

A small summary of:
- how many cases could be matched in the DB, and based on which columns
- how many cases were “correctly” revised, in terms of upcoding (still need to do after the `grouper` is available for python)

In [14]:
if revised_case_db.shape[0] <= len(revised_case_id):
    print(f'Based on the columns "case_id", "gender, "age_years", "duration_of_stay", "pccl", "drg", there are {num_match_case_id} out of {len(revised_case_id)} revised cases from DtoD that can be matched with the bfs database data for {file_info.hospital_name_db} {file_info.year}' )
else:
    print(f'Based on the columns "case_id" and "Patient_id", "gender, "age_years", "duration_of_stay", "pccl", "drg", there are {num_match_case_patient_id} out of {len(revised_case_id)} revised cases from DtoD that can be matched with the bfs database data for {file_info.hospital_name_db} {file_info.year}' )

Based on the columns "case_id" and "Patient_id", "gender, "age_years", "duration_of_stay", "pccl", "drg", there are 146 out of 150 revised cases from DtoD that can be matched with the bfs database data for USZ 2019


In [15]:
print('All the normalized revised cases are exported to here: {}'.format(output_path))

All the normalized revised cases are exported to here: /home/jovyan/work/src/revised_case_normalization/normalized_revision_data/USZ_2019.csv
